In [2]:
# Data processing and plotting libraries
import datetime
import random
import matplotlib.pyplot as plt
import matplotlib
import xarray as xr
import numpy as np
import pandas as pd
import cartopy.crs as ccrs
from sklearn.neighbors import KernelDensity
from matplotlib.pyplot import figure
import seaborn as sns
import xarray.ufuncs as xu 

# File system interface
import s3fs
import fsspec

# Dask based imports
import dask
from dask.distributed import performance_report, Client, progress

# Garbage/debugging and worker allocation
import gc
from distributed import WorkerPlugin

# set up local cluster on laptop for Dask
client = Client()  
client

/Users/seanbratton/opt/anaconda3/lib/python3.9/site-packages/distributed/node.py:180: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54760 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:54760/status,
Dashboard: http://127.0.0.1:54760/status,Workers: 5
Total threads: 10,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54769,Workers: 5
Dashboard: http://127.0.0.1:54760/status,Total threads: 10
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:55289,Total threads: 2
Dashboard: http://127.0.0.1:55301/status,Memory: 6.40 GiB
Nanny: tcp://127.0.0.1:54788,


In [3]:
# Create a class object
class WorkerExplicitGC(WorkerPlugin):
    def setup(self, worker):
        self.worker = worker
        gc.disable()             

    def transition(self, key, start, finish, *args, **kwargs):
       if finish == 'executing':
          self.worker._throttled_gc.collect()
          if gc.isenabled():
             gc.disable()

plugin = WorkerExplicitGC()
client.register_worker_plugin(plugin)

{'tcp://127.0.0.1:55284': {'status': 'OK'},
 'tcp://127.0.0.1:55286': {'status': 'OK'},
 'tcp://127.0.0.1:55287': {'status': 'OK'},
 'tcp://127.0.0.1:55288': {'status': 'OK'},
 'tcp://127.0.0.1:55289': {'status': 'OK'}}

In [ ]:
################# QUESTION 1 MATERIALS #########################

# Create a dataset that contains the monthly means of Sea Surface Temperature anomalies and precipitation 
# anomalies from Jan 1979-Dec 2021 over the Pacific Basin (65°N to 65°S, 120°E to 60°W)

In [ ]:
###### SST Analysis ########

In [16]:
# https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/1979/e5.moda.an.sfc.128_034_sstk.ll025sc.1979010100_1979120100.nc

## Load in SST monthly data from UCAR

base_url = 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/'
base_url2 = '/e5.moda.an.sfc.128_034_sstk.ll025sc.'
base_url3 = '.nc'

# period of interest - pd.date_range allos you pick range of data
pr = pd.date_range(start='1979-01',end='2021-12', freq='AS')

file_list=[]
for dt in pr:
    # get recent year and month
    year = dt.strftime('%Y')
    month = dt.strftime('%Y%m%d%H')
    month2 = (dt + pd.DateOffset(months=11)).strftime('%Y%m%d%H')

    # build complete file name
    single_file=(base_url+year+base_url2+month+'_'+month2+base_url3)
    file_list.append(single_file)

In [18]:
 # Create SST xarray dataset from files aggregated
 
 ds_sst = xr.open_mfdataset(file_list, parallel=True)

distributed.worker - WARNING - Compute Failed
Function:  execute_task
args:      ((<function apply at 0x7faef81065e0>, <function open_dataset at 0x7faf40ee0e50>, ['https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/2013/e5.moda.an.sfc.128_034_sstk.ll025sc.2013010100_2013120100.nc'], (<class 'dict'>, [['engine', None], ['chunks', (<class 'dict'>, [])]])))
kwargs:    {}
Exception: "OSError(-90, 'NetCDF: file not found')"

distributed.worker - WARNING - Compute Failed
Function:  execute_task
args:      ((<function apply at 0x7faef81065e0>, <function open_dataset at 0x7faf40ee0e50>, ['https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/1997/e5.moda.an.sfc.128_034_sstk.ll025sc.1997010100_1997120100.nc'], (<class 'dict'>, [['engine', None], ['chunks', (<class 'dict'>, [])]])))
kwargs:    {}
Exception: "OSError(-90, 'NetCDF: file not found')"

distributed.worker - WARNING - Compute Failed
Function:  execute_task
args:      ((<function apply at 0x7f91e00865e0

OSError: [Errno -90] NetCDF: file not found: b'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.an.sfc/2013/e5.moda.an.sfc.128_034_sstk.ll025sc.2013010100_2013120100.nc'

In [ ]:
# save observed SST dataset as a netCDF

ds_sst.to_netcdf(path = '/Users/seanbratton/documents/UIUC/ATMS523/module_4/sst_observed_dataset.nc')


In [ ]:
# Calculate the SSt climo over each month and then average the months togetehr
climo_sst= ds_sst['SSTK'].sel(latitude=slice(65,-65),longitude=slice(120,360-60)).groupby('time.month').mean(dim='time')

# take each data point for each month and subtract from the monthly mean for the month the
# day is in. 
anom_sst = (ds_sst.groupby('time.month') - climo_sst)

In [ ]:
# save SST anomaly dataset as a netCDF

anom_sst.to_netcdf(path = '/Users/seanbratton/documents/UIUC/ATMS523/module_4/sst_anom_dataset.nc')

In [ ]:
###### Precipitation Analysis ########

In [15]:
## Load in Precip monthly data from UCAR

base_url = 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.fc.sfc.meanflux/'
base_url2 = '/e5.moda.fc.sfc.meanflux.235_030_mcpr.ll025sc.'
base_url3 = '.nc'

# period of interest
pr = pd.date_range(start='1979-01',end='2021-12', freq='AS')

file_list=[]
for dt in pr:
    # get recent year and month
    year = dt.strftime('%Y')
    month = dt.strftime('%Y%m%d%H')
    month2 = (dt + pd.DateOffset(months=11)).strftime('%Y%m%d%H')

    # build complete file name
    single_file=(base_url+year+base_url2+month+'_'+month2+base_url3)
    file_list.append(single_file)


import xarray as xr
ds_cpr = xr.open_mfdataset(file_list, parallel=True, engine='netcdf4').\
    sel(latitude=slice(65,-65),longitude=slice(120,360-60))

## Load in Meanflux monthly data from UCAR

base_url = 'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.fc.sfc.meanflux/'
base_url2 = '/e5.moda.fc.sfc.meanflux.235_029_mlspr.ll025sc.'
base_url3 = '.nc'

# period of interest
pr = pd.date_range(start='1979-01',end='2021-12', freq='AS')

file_list=[]
for dt in pr:
    # get recent year and month
    year = dt.strftime('%Y')
    month = dt.strftime('%Y%m%d%H')
    month2 = (dt + pd.DateOffset(months=11)).strftime('%Y%m%d%H')

    # build complete file name
    single_file=(base_url+year+base_url2+month+'_'+month2+base_url3)
    file_list.append(single_file)


import xarray as xr
ds_lspr = xr.open_mfdataset(file_list, parallel=True, engine='netcdf4').\
    sel(latitude=slice(65,-65),longitude=slice(120,360-60))

ds_pr = ds_lspr['MLSPR'] + ds_cpr['MCPR']

distributed.worker - WARNING - Compute Failed
Function:  execute_task
args:      ((<function apply at 0x7f91e00865e0>, <function open_dataset at 0x7f91f0f5f940>, ['https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.fc.sfc.meanflux/2012/e5.moda.fc.sfc.meanflux.235_030_mcpr.ll025sc.2012010100_2012120100.nc'], (<class 'dict'>, [['engine', 'netcdf4'], ['chunks', (<class 'dict'>, [])]])))
kwargs:    {}
Exception: "OSError(-90, 'NetCDF: file not found')"

distributed.worker - WARNING - Compute Failed
Function:  execute_task
args:      ((<function apply at 0x7f91e00865e0>, <function open_dataset at 0x7f91f0f5f940>, ['https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.fc.sfc.meanflux/2006/e5.moda.fc.sfc.meanflux.235_030_mcpr.ll025sc.2006010100_2006120100.nc'], (<class 'dict'>, [['engine', 'netcdf4'], ['chunks', (<class 'dict'>, [])]])))
kwargs:    {}
Exception: "OSError(-90, 'NetCDF: file not found')"

distributed.worker - WARNING - Compute Failed
Function:  execute_task


OSError: [Errno -90] NetCDF: file not found: b'https://rda.ucar.edu/thredds/dodsC/files/g/ds633.1_nc/e5.moda.fc.sfc.meanflux/2006/e5.moda.fc.sfc.meanflux.235_030_mcpr.ll025sc.2006010100_2006120100.nc'

In [ ]:
# save Precip observed dataset as a netCDF

dr_pr.to_netcdf(path = '/Users/seanbratton/documents/UIUC/ATMS523/module_4/precip_orig_dataset.nc')


In [ ]:
# Calculate the SSt climo over each month and then average the months togetehr
climo_precip= ds_pr.groupby('time.month').mean(dim='time')

# take each data point for each month and subtract from the monthly mean for the month the
# day is in.
anom_precip = (ds_pr.groupby('time.month') - climo_precip)

In [ ]:
# save precip anomaly dataset as a netCDF

anom_precip.to_netcdf(path = '/Users/seanbratton/documents/UIUC/ATMS523/module_4/precip_anom_dataset.nc')


In [ ]:
############## QUESTION 2 MATERIALS #################

# From the dataset shown in 1, deseasonalize (remove the mean monthly anomaly from the annual 
# mean from each point), detrend, and standardize the SST anomalies. Standardization means subtract 
# the long term mean and divide by the standard deviation,

In [ ]:
###### SST Standardization ##########

In [ ]:
# SST - Deseaonalize data

climatology_mean = ds_sst['SSTK'].sel(latitude=slice(65,-65),longitude=slice(120,360-60)).groupby("time.month").mean("time")
climatology_std = ds_sst['SSTK'].sel(latitude=slice(65,-65),longitude=slice(120,360-60)).groupby("time.month").std("time")
stand_anomalies = xr.apply_ufunc(
    lambda x, m, s: (x - m) / s,
    ds_sst['SSTK'].sel(latitude=slice(65,-65),longitude=slice(120,360-60)).groupby("time.month"),
    climatology_mean,
    climatology_std,
    dask="parallelized"
)


In [ ]:
# SST - detrended data

def detrend_dim(da, dim, deg=1):
    # detrend along a single dimension
    # least squares polynomial fit
    p = da.polyfit(dim=dim, deg=deg)
    print(p)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - fit

# lt_mean_sst= ds_sst['SSTK'].sel(latitude=slice(65,-65),longitude=slice(120,360-60))

detrend_sst = detrend_dim(stand_anomalies,'time',1)


In [ ]:
# save standarized SST locally as NetCDF

detrend_sst.to_netcdf(path = '/Users/seanbratton/documents/UIUC/ATMS523/module_4/sst_standardized_dataset_v2.nc')


In [ ]:
#### Precipitation Standardization

In [ ]:
# Precip - Deseaonalize data

climatology_mean_precip = ds_tp['TP'].sel(latitude=slice(65,-65),longitude=slice(120,360-60)).groupby("time.month").mean("time")
climatology_std_precip = ds_tp['TP'].sel(latitude=slice(65,-65),longitude=slice(120,360-60)).groupby("time.month").std("time")
stand_anomalies_precip = xr.apply_ufunc(
    lambda x, m, s: (x - m) / s,
    ds_tp['TP'].sel(latitude=slice(65,-65),longitude=slice(120,360-60)).groupby("time.month"),
    climatology_mean_precip,
    climatology_std_precip,
    dask="parallelized"
)

In [ ]:
# Precip - detrended data

def detrend_dim(da, dim, deg=1):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=deg)
    print(p)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - fit

# lt_mean_sst= ds_sst['SSTK'].sel(latitude=slice(65,-65),longitude=slice(120,360-60))

detrend_precip = detrend_dim(stand_anomalies_precip,'time',1)


In [ ]:
# save standarized Precip locally as NetCDF

detrend_precip.to_netcdf(path = '/Users/seanbratton/documents/UIUC/ATMS523/module_4/precip_standardized_dataset.nc')
